# <center>Big Data &ndash; Exercises</center>
## <center>Fall 2018 &ndash; Week 6 &ndash; ETH Zurich</center>
## <center>YARN + Spark</center>

Reading: 
-  Matei Zaharia, Holden Karau, Andy Konwinski, Patrick Wendell (2015). Learning Spark. O’Reilly Media, Inc.  [ETH library] (Chapters 3,4: mandatory; 5,6,8 optional)
-  White, T. (2015). Hadoop: The Definitive Guide (4th ed.). O’Reilly Media, Inc. [ETH library] (Chapters 4: mandatory)
-  Recommended: Vinod Kumar Vavilapalli et al. (2013) Apache Hadoop YARN: Yet Another Resource Negotiator [link](https://dl.acm.org/citation.cfm?id=2523633)
-  Recommended: Matei Zaharia et al. (2012) Resilient Distributed Datasets: A Fault-Tolerant Abstraction for In-Memory Cluster Computing [link](https://www.usenix.org/node/162809)
 

This exercise will consist of 2 main parts: 
* Architecture and theory of YARN
* Hands-on practice with Spark on Azure HDInsight

# Exercise 1 &mdash; What is YARN?

Fundamentally, “Yet Another Resource Negotiator,” **YARN**  is a resource scheduler designed to work on existing and new Hadoop clusters. 

### 1.1 &ndash; List at least 3 main shortcomings of MapReduce v1, which are addressed by YARN design.

### Solution
1. Scalability. Scale to 10,000 nodes.
2. Need for non-MapReduce workloads. Ability to share cluster with MPI, graph processing, and any user code.
3. Better resource utilization. Problem with idle reducers, when mappers are not finished (and vice versa). The entire system should be used at maximum capacity when needed.
4. Agility. Ability to maintain MapReduce frameworks of different versions.

### 1.2 &ndash; State which of the following statements are true:

1. The ResourceManager has to provide fault tolerance for resources across the cluster 

1. Container allocation/deallocation can take place in a dynamic fashion as the application progresses. 

1. An application can only request resources in terms of memory usage and number of CPUs. 

1. Communications between the ResourceManager and NodeManagers are heartbeat-based. 

1. The ResourceManager does not have a global view of all usage of cluster resources. Therefore, it tries to make better scheduling decisions based on probabilistic prediction. 

1. ResourceManager has the ability to request resources back from a running application.

### Solution

1. [False]

1. [True]

1. [False] 

1. [True]

1. [False]

1. [True]

### 1.3 &ndash; Whose responsibility is it? Say which component of YARN is resposible for each of the following tasks.

1. Fault Tolerance of running applications *[ResourceManager | ApplicationMaster | NodeManager ]*
1. Asking for resources needed for an application *[ResourceManager | ApplicationMaster | NodeManager ]*

1. Providing leases to use containers *[ResourceManager | ApplicationMaster | NodeManager]*

1. Tracking status and progress of running applications *[ResourceManager | ApplicationMaster | NodeManager]*





### Solution

1. Fault Tolerance of running applications *[ResourceManager | **ApplicationMaster** | NodeManager ]*

1. Asking for resources needed for an application *[ResourceManager | **ApplicationMaster** | NodeManager ]*

1. Providing leases to use containers *[**ResourceManager** | ApplicationMaster | NodeManager]*

1. Tracking status and progress of running applications *[ResourceManager | **ApplicationMaster** | NodeManager]*



### 1.4 &ndash; What is the typical configuration for YARN? Choose for the following components how many instances of them there are in a cluster.

```
1. ResourceManager                  a. One per cluster

2. ApplicationMaster                b. One per node

3. NodeManager                      c. Many per cluster, but not per every node

4. Container                        d. Many per node 
```


### Solution
1 - a  
2 - c  
3 - b  
4 - d  

Note that we can have multiple ApplicationMasters on the same node as long as they belong to different applications and none of them is aware about the presence of the other ApplicationMasters.

#  CapacityScheduler.

The CapacityScheduler is a pluggable scheduler for YARN which allows for multiple tenants to securely share a large cluster such that their applications are allocated resources in a timely manner under constraints of allocated capacities. The CapacityScheduler is designed to maximize the throughput and the utilization of the cluster.

Consider the following hierarchical queues configuration for CapacityScheduler with the DefaultResourceCalculator, which takes into account only memory usage.


![img](https://ethbigdata2017.blob.core.windows.net/exercise06/schedule.png)

### 1.5 &ndash;  Calculate configured capacities of all leaves, assuming a cluster of 100 nodes, each with 10 GB of memory allocated for YARN containers

### Solution
Total capacity = 100 nodes x 10 GB = 1000 GB   
Spark = 1000 GB x 0.4 = 400 GB   
Jack = 400 GB x 0.3 = 120 GB   
Phill = 400 GB x 0.7 = 280 GB  
MPI = 1000 GB x 0.1 = 100 GB  
Jim's lab = 1000 GB x 0.5 = 500 GB 

### 1.6 &ndash; Consider the following timeline of events happening in the cluster:

#### Scenario 1

1. In the beginning, the entire cluster is free, with no application in any queues. User A submits 3 applications to queue *Jack* with 120 GB of required memory each. Assume each application takes 1 hour to finish. After how much time will all 3 application be finished and why?

2. 30 minutes after the user A submitted their jobs, user B submits an application to queue *Phill* with resource requirements of 280GB. Assuming that the task takes 2 hours to complete and preemption is disabled, when will all jobs be finished?

#### Scenario 2

1. In the beginning, the entire cluster is free, with no application in any queues. User A submits 10 applications to queue *Jack* with 100 GB of required memory each. Assume each application takes 1 hour to finish. After how much time will all 10 application be finished and why?

2. 30 minutes after user A submitted their jobs, user B submits an application to queue *Phill* with resource requirements of 280GB. Assuming that the task takes 2 hours to complete and preemption is disabled, when will all jobs be finished?

### Solution
Scenario 1.
1. All three aplications will be launched at the same time regardless capacity of the queue, since there are enough cluster resources to run all three jobs together ($120*3 \le 1000$).
2. In 30 minutes, the job of user B  will also start executing since the cluster will have $1000-120*3 = 640 GB$ of free resources. Therefore, the whole execution will take 2.5 hours.


Scenario 2.
1. All 10 aplications will be launched at the same time regardless capacity of the queue, since there are enough cluster resources to run all 10 jobs together ($100 * 10 \le 1000$).
2. In 30 minutes, the job of the user B  will arrive and get inserted in a waiting queue, since there will be no resources to start the job. In 1 hour all cluster resources will become available and the task of user B will start being executed. Therefore, the whole execution will take 3 hours

### 1.7 &ndash; Assuming a cluster of 10 nodes, each with 16 GB of memory and 2 cores allocated for YARN containers.  For the same CapacityScheduler configuration but with the DominantResourceCalculator (takes into account memory usage and CPUs), consider the following timeline of events happening in the cluster:



#### Scenario 1

1. In the beginning, the entire cluster is free, with no application in any queues. User A submits 3 applications to queue *Jack* with *(80 GB, 10 CPUs)* resource requirements each. Assume each application takes 1 hour to finish. 

2. 30 minutes after the user A submitted their jobs, user B submits an application to queue *Phill* with resource requirements of *(580 GB, 10 CPUs)*. Assuming that the task takes 2 hours to complete and preemption is disabled, when will all jobs be finished?

#### Scenario 2

1. In the beginning, the entire cluster is free, with no applications in any queues. User A submits 2 applications to queue *Jack* with *(60 GB, 6 CPUs)* resource requirements each. In addition, user B submits 2 applications to queue *Phill* with resource requirements of *(10 GB, 10 CPUs)* each. Assume each application takes 1 hour to finish and preemption is disabled, when will all jobs be finished?


#### Scenario 3

1. In the beginning, the entire cluster is free, with no applications in any queues. User A submits 1 application to queue *Jack* with *(1 GB, 20 CPUs)* resource requirements. Assume that the application runs for 1 hour.

2. 30 minutes after the user A submitted their jobs, user B submits an application to queue *Phill* with resource requirements of *(100 GB, 1 CPU)*. Assuming that the task takes 2 hours to complete and preemption is disabled, when will all jobs be finished?


### Solution
The cluster has total capacity 160GB and 20 CPUs.
	 

 Note. By default, all queues in the capacity scheduler have *user-limit-factor* set to 1 which ensures that a single user can never take more than the queue’s configured capacity irrespective of how idle the cluster is. Therefore, if all queues are configured with the default user-limit-factor, then all jobs will be rejected in all discussed scenarios. Below we assume that the size of a single job is not limited by the *user-limit-factor*. 

Scenario 1.
1. First, two aplications will be launched at the same time, regardless of the capacity of the queue, since there are enough cluster resources to run two jobs together (80 GB,10 CPUs) * 2 <= (160 GB, 20 CPUs). The third job will inserted in the waiting queue.

2. After 30 minutes, the job of user B  will be **rejected**, since the cluster cannot satisfy its requirements.

Jobs of the client A will be completed within 2 hours.


Scenario 2.
1. First, we calculate the dominant resource:
(60 GB, 6 CPUs) -> (37.5%,30%) -> (**37.5%**,30%)  
(10 GB, 10 CPUs) -> (6.25%,50%) -> (6.25%,**50%**)    

Since the target capacity is 30:70, we will have the following scenario

1. [10,10] Start B.1 at t=0
2. [70,16] Start A.1 at t=0
3. [0,0] A.1 and B.1 are completed at t=1h
4. [10,10] Start B.2 at t=1h
5. [70,16] Start A.2 at t=1h
6. [0,0] A.2 and B.2 are completed at t=2h

In more detail: 
- At first, both A and B own 0% of each resource. 
- We know that the dominant resource of A is 37.5% memory and the dominant resource of B is 50% CPUs. 
- At t=0 both clients use 0% of their dominant share. 
- In order to solve this tie the first allocation can go to B because it is below its instantaneous fair share (meaning that its dominant resource utilizes less than 70% which is its share in the Spark queue in contrast with A that wants more than 30% which is its share in the queue). 
- Now A owns 0/30=0% of its fair share of memory (its dominant resource) and B owns 50/70 = 71% of its fair share of CPUs (its dominant resource).
- The overall usage of the cluster is 10 of 160 GB and 10 of 20 CPUs. 
- Thus the next allocation goes to A because it has the least share of its dominant resource (0% vs. 71%). 
- Now A owns 37.5% of the memory and B owns 50% of the CPUs. By putting fair share weights A owns 37.5/30 = 125% of its fair share of memory and B owns 71% of its fair share of the CPUs.
- Overall usage: 70 of 160 GB and 16 of 20 CPUs. The next allocation should go to B (because its fair share weight is less than the one of A - 125% vs. 71% ) but there are not enough resources.
- There aren't also enough resources for A either. 
- The algorithm terminates and both A and B get one container. 
- One hour later, the same scenario happens again.

Scenario 3.
It will take 3 hours, since resources for the job of the user B will become available at t = 1h.



## 2. Setup the Spark cluster in Azure

### Create a cluster

- Sign into the azure portal (portal.azure.com).
- Click "+" in the upper left corner.
- Search for "HDInsight".
- Give the cluster a unique name.
- In the "Select Cluster Type" choose **Spark** and a standard Cluster Tier (Finish with pressing "select").
- Set up a Spark cluster with default configuration.
- Wait for 20 mins so that your cluster is ready.

### Access your cluster

Make sure you can access your cluster (the NameNode) via SSH:

```
$ ssh <ssh_user_name>@<cluster_name>-ssh.azurehdinsight.net
```

If you are using Linux or MacOSX, you can use your standard terminal.
If you are using Windows you can use:
- Putty SSH Client and PSCP tool (get them at [here](http://www.chiark.greenend.org.uk/~sgtatham/putty/download.html)).
- This Notebook server terminal (Click on the Jupyter logo and the goto New -> Terminal).
- Azure Cloud Terminal (see the HBase exercise sheet for details)


The cluster has its own Jupyter server. We will use it. You can access it through the following link:
```
 https://<cluster_name>.azurehdinsight.net/jupyter
```

You can access cluster's YARN in your browser
```
 https://<cluster_name>.azurehdinsight.net/yarnui/hn/cluster
```

The Spark UI can be accessed via Azure Portal, see [Spark job debugging](https://docs.microsoft.com/en-us/azure/hdinsight/hdinsight-apache-spark-job-debugging)

# <span style="color:red">You need to upload this notebook to your cluster's Jupyter in order to execute Python code blocks.</span>
To do this, just open the Jupyter through the link given above and use the "Upload" button.


## 3. Apache Spark Architecuture

Spark is a cluster computing platform designed to be fast and general purpose. Spark extends the MapReduce model to efficiently cover a wide range of workloads that previously required separate distributed systems, including interactive queries and stream processing. Spark offers the ability to run computations in memory.

At a high level, every Spark application consists of a **driver program** that launches various parallel operations on a cluster. The driver program contains your application's main function and defines distributed datasets on the cluster, then applies operations to them.

Driver programs access Spark through a **SparkContext** object, which represents a connection to a computing cluster. There is no need to create a SparkContext; it is created for you automatically when you run the first code cell in the Jupyter

The driver communicates with a potentially large number of distributed workers called **executors**. The driver runs in its own process and each executor is a separate process. A driver and its executors are together termed a Spark application.

![Image of Account](http://spark.apache.org/docs/latest/img/cluster-overview.png)

### 3.1 Understand resilient distributed datasets (RDD)

An RDD in Spark is simply an immutable distributed collection of objects. Each RDD is split into multiple partitions, which may be computed on different nodes of the cluster. 

##### What are RDD operations?
RDDs offer two types of operations: **transformations** and **actions**.

* **Transformations** create a new dataset from an existing one. Transformations are lazy, meaning that no transformation is executed until you execute an action.
* **Actions** compute a result based on an RDD, and either return it to the driver program or save it to an external storage system (e.g., HDFS)




Transformations and actions are different because of the way Spark computes RDDs. Although you can define new RDDs any time, Spark computes them only in a **lazy** fashion, that is, the first time they are used in an action.

##### How do I make an RDD?

RDDs can be created from stable storage or by transforming other RDDs. Run the cells below to create RDDs from the sample data files available in the storage container associated with your Spark cluster. One such sample data file is available on the cluster at `wasb:///example/data/fruits.txt`. 

In [ ]:
fruits = sc.textFile('wasb:///example/data/fruits.txt')
yellowThings = sc.textFile('wasb:///example/data/yellowthings.txt')

##### RDD transformations
Following are examples of some of the common transformations available. For a detailed list, see [RDD Transformations](https://spark.apache.org/docs/2.0.0/programming-guide.html#transformations)

Run some transformations below to understand this better.



In [ ]:
# map
fruitsReversed = fruits.map(lambda fruit: fruit[::-1])

In [ ]:
# filter
shortFruits = fruits.filter(lambda fruit: len(fruit) <= 5)

In [ ]:
# flatMap
characters = fruits.flatMap(lambda fruit: list(fruit))

In [ ]:
# union
fruitsAndYellowThings = fruits.union(yellowThings)

In [ ]:
# intersection
yellowFruits = fruits.intersection(yellowThings)

In [ ]:
# distinct
distinctFruitsAndYellowThings = fruitsAndYellowThings.distinct()

In [ ]:
# groupByKey
yellowThingsByFirstLetter = yellowThings.map(lambda thing: (thing[0], thing)).groupByKey()

In [ ]:
# reduceByKey
numFruitsByLength = fruits.map(lambda fruit: (len(fruit), 1)).reduceByKey(lambda x, y: x + y)

##### RDD actions
Following are examples of some of the common actions available. For a detailed list, see [RDD Actions](https://spark.apache.org/docs/2.0.0/programming-guide.html#actions).

Run some transformations below to understand this better. 

In [ ]:
# collect
fruitsArray = fruits.collect()
yellowThingsArray = yellowThings.collect()
print(fruitsArray)
print(yellowThingsArray)

In [ ]:
# count
numFruits = fruits.count()
numFruits

In [ ]:
# take
first3Fruits = fruits.take(3)
first3Fruits

In [ ]:
# reduce
letterSet = fruits.map(lambda fruit: set(fruit)).reduce(lambda x, y: x.union(y))
letterSet

##### Lazy evaluation
Lazy evaluation means that when we call a transformation on an RDD (for instance, calling `map()`), the operation is not immediately performed. Instead, Spark internally records metadata to indicate that this operation has been requested. Rather than thinking of an RDD as containing specific data, it is best to think of each RDD as
consisting of instructions on how to compute the data that we build up through transformations. Loading data into an RDD is lazily evaluated in the same way transformations are. So, when we call `sc.textFile()`, the data is not loaded until it is necessary. As with transformations, the operation (in this case, reading the data) can
occur multiple times.


Finally, as you derive new RDDs from each other using transformations, Spark keeps track of the set of dependencies between different RDDs, called the lineage graph. For instance, the code bellow corresponds to the following graph:

![graph](https://ethbigdata2017.blob.core.windows.net/exercise06/dag.png)



In [ ]:
apples = fruits.filter(lambda x: "apple" in x)
lemons = yellowThings.filter(lambda x: "lemon" in x)
applesAndLemons = apples.union(lemons)
print(applesAndLemons.collect())
print(applesAndLemons.toDebugString())

### 3.2 Exercise

1. What does the code below do?
1. Draw the linage graph for the code
1. List actions and transformations used in it
1. When are all calculations executed?
1. If we call `result.collect()` again, what will Spark do to perform the action? 

In [ ]:
text = sc.textFile('wasb:///example/data/fruits.txt')
words = text.flatMap(lambda x: x.split(" "))
result = words.map(lambda x: (x, 1)).reduceByKey(lambda x, y: x + y)
result.saveAsTextFile('wasb:///example/data/result.txt')
result.collect()

**Solution**
1. It is word counting.
1. result.toDebugString
1. Transformations: flatMap, map, reduceByKey. Actions: saveAsTextFile, collect
1. When saveAsTextFile is called
1. It will recalculate everything again, including reading the text file.

### 3.3 Persistence (Caching)

Spark's RDDs are by default recomputed each time you run an action on
them. If you would like to reuse an RDD in multiple actions, you can ask Spark to persist it using `RDD.persist()`. After computing it the first time, Spark will store the RDD contents in memory (partitioned across the machines in your cluster), and reuse them in future actions. Persisting RDDs on disk instead of memory is also possible.

If you attempt to cache too much data to fit in memory, Spark will automatically evict old partitions using a Least Recently Used (LRU) cache policy. For the memory-only storage levels, it will recompute these partitions the next time they are accessed,
while for the memory-and-disk ones, it will write them out to disk. In either case, this means that you don't have to worry about your job breaking if you ask Spark to cache too much data. However, caching unnecessary data can lead to eviction of useful data
and more recomputation time. Finally, RDDs come with a method called `unpersist()` that lets you manually remove them from the cache.


### 3.4 Exercise:
1. Write some code which can benefit from caching.
1. Where should we ask Spark to persist the RDD in Exercise 3.2 to prevent it from re-executing the code when we call `collect()` again?

In [ ]:
# Solution 1
fruits.persist()
numFruits = fruits.count()
first3Fruits = fruits.take(3)
fruits.unpersist()
print(numFruits,first3Fruits)

In [ ]:
# Solution 2
text = sc.textFile('wasb:///example/data/fruits.txt')
words = text.flatMap(lambda x: x.split(" "))
result = words.map(lambda x: (x, 1)).reduceByKey(lambda x, y: x + y).persist()
result.saveAsTextFile('wasb:///example/data/result.txt')
result.collect()

### 3.5 Working with Key/Value Pairs


Spark provides special operations on RDDs containing key/value pairs. These RDDs
are called *pair RDDs*. Pair RDDs are a useful building block in many programs, as
they expose operations that allow you to act on each key in parallel or regroup data
across the network. For example, pair RDDs have a `reduceByKey()` method that can
aggregate data separately for each key, and a `join()` method that can merge two
RDDs together by grouping elements with the same key. Pair RDDs are also still RDDs. 



In [ ]:
# Example
rdd = sc.parallelize([("key1", 0) ,("key2", 3),("key1", 8) ,("key3", 3),("key3", 9)])
rdd2 = rdd.mapValues(lambda x: (x, 1)).reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))
print(rdd2.collect())
print(rdd2.toDebugString())

### 3.6 Exercise
1. What does the code above do? 
2. Where can it be used? Complete the code to perform the desired functionality. 


In [ ]:
# Solution
# 1. it accumulates the values of items and also estimate the number of accumulated elements
# 2. It can be used for estimating the average value of every item in the dataset.

rdd = sc.parallelize([("key1", 0) ,("key2", 3),("key1", 8) ,("key3", 3),("key3", 9)])
rdd2 = rdd.mapValues(lambda x: (x, 1)).reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))
average = rdd2.mapValues(lambda x: x[0]/x[1])
print(average.collect())

### 3.7 Spark Partitioning
Spark programs can choose to control their RDDs' partitioning
to reduce communication. Partitioning will not be helpful in all applications, for
example, if a given RDD is scanned only once, there is no point in partitioning it in
advance. It is useful only when a dataset is reused multiple times in key-oriented
operations such as joins.

Spark's partitioning is available on all RDDs of key/value pairs, and causes the system
to group elements based on a function of each key. Although Spark does not give
explicit control of which worker node each key goes to (partly because the system is
designed to work even if specific nodes fail), it lets the program ensure that a set of
keys will appear together on some node.


Many of Spark's operations involve shuffling data by key across the network. All of
these will benefit from partitioning. Examples of operations that benefit from
partitioning are `cogroup()`, `groupWith()`, `join()`, `leftOuterJoin()`, `rightOuterJoin()`, `groupByKey()`, `reduceByKey()`, `combineByKey()`, and `lookup()`.

By default PySpark uses hash partitioning as the partitioning function. A way to define a custom partition is by using the function `partitionBy()`. To use `partitionBy()` the RDD must consist of tuple objects. This function is a transformation, therefore a new RDD will be returned. In the following example we are going to see a default partitioning scheme of Spark as well as a custom partitioning.

In [ ]:
nums = [(1, 1), (2, 2), (3, 3)]

In [ ]:
pairs = sc.parallelize(nums)

In [ ]:
print("Number of partitions: {}".format(pairs.getNumPartitions()))
print("Partitions structure: {}".format(pairs.glom().collect()))

Let's try to define a custom partitioning now.

In [ ]:
pairs = sc.parallelize(nums).partitionBy(2)

In [ ]:
print("Number of partitions: {}".format(pairs.getNumPartitions()))
print("Partitions structure: {}".format(pairs.glom().collect()))

### 3.8 Converting a user program into tasks

A Spark driver is responsible for converting a user program into units of physical execution called tasks. At a high level, all Spark programs follow the same structure: they create RDDs from some input, derive new RDDs from those using transformations, and perform actions to collect or save data. A Spark program implicitly creates a logical **directed acyclic graph (DAG)** of operations.
When the driver runs, it converts this logical graph into a physical execution plan.

Spark performs several optimizations, such as "pipelining" map transformations together to merge them, and converts the execution graph into a set of **stages**.
Each stage, in turn, consists of multiple tasks. The tasks are bundled up and prepared to be sent to the cluster. Tasks are the smallest unit of work in Spark; a typical user program can launch hundreds or thousands of individual tasks.

Each RDD maintains a pointer to one or more parents along with metadata about what
type of relationship they have. For instance, when you call `val b = a.map()` on an
RDD, the RDD `b` keeps a reference to its parent `a`. These pointers allow an RDD to be
traced to all of its ancestors.

The following phases occur during Spark execution:
* User code defines a DAG (directed acyclic graph) of RDDs. Operations on RDDs create new RDDs that refer back to their parents, thereby creating a graph.
* Actions force translation of the DAG to an execution plan. When you call an action on an RDD, it must be computed. This requires computing its parent RDDs as well. 
* Spark's scheduler submits a job to compute all needed RDDs. That job will have one or more stages, which are parallel waves of computation composed of tasks. Each stage will correspond to one or more RDDs in the DAG. A single stage can correspond to multiple RDDs due to pipelining.
* Tasks are scheduled and executed on a cluster
* Stages are processed in order, with individual tasks launching to compute segments of the RDD. Once the final stage is finished in a job, the action is complete.

If you visit the application's web UI, you will see how many stages occur in order to
fulfill an action. The Spark UI can be accessed via Azure Portal, see [Spark job debugging](https://docs.microsoft.com/en-us/azure/hdinsight/hdinsight-apache-spark-job-debugging)

### 3.9 Exercise. 

1. Why is Spark faster than Hadoop MapReduce?
1. Study the examples above via Spark UI. Observe how many stages they have. 
1. Which of the graphs below are DAGs?
![graph](https://ethbigdata2017.blob.core.windows.net/exercise06/dags.png)

### Solution:
1. There are many reasons for that. Firstly, Spark processes data in-memory while Hadoop MapReduce persists back to the disk after a map or reduce action. Secondly, Spark pipelines transformations to merge them into stages. 
2. Follow the following [instruction](https://docs.microsoft.com/en-us/azure/hdinsight/hdinsight-apache-spark-job-debugging)
3. DAGs: 2,3,4,5,7

### 3.10 True or False
Say if the following statements are *true* or *false*, and explain why.

1. Each RDD is split into multiple partitions, which may be computed on different nodes of the cluster.
1. Transformations construct a new RDD from a previous one and immediately calculate the result
1. Spark's RDDs are by default recomputed each time you run an action on them
1. After computing an RDD, Spark will store its contents in memory and reuse them in future actions.
1. When you derive new RDDs using transformations, Spark keeps track of the set of dependencies between different RDDs.

Solution
1. True
1. False. Spark will not begin to execute until it sees an action.
1. True
1. False. Users have to use persist() or cache() for that.
1. True

## 4. TF-IDF in Spark
In this exercise you will implement a simple query engine over the Gutenberg dataset using Spark.
The [Gutenberg dataset](http://web.eecs.umich.edu/~lahiri/gutenberg_dataset.html) consists of 3036 free ebooks. The goal of this exercise is to develop a search engine to find the most relevant books given a text query.

### 4.1 Get the data
You can download the dataset from https://ethbigdata2017.blob.core.windows.net/exercise06/Gutenberg.zip
- Create a Spark cluster as explained in the exercise of last week.
- When ready, you can upload the dataset into its Blob Storage container. You can either 1) connect via `SSH`, `wget` the dataset, decompress it and upload it to HDFS, or 2) download the dataset on your computer, decompress it locally and upload the dataset folder to the Blob Storage using [Azure Storage Explorer](http://storageexplorer.com/). Before uploading to HDFS run the following command inside the folder where the txt files are, in order name them by their numerical ordering: ```ls | cat -n | while read n f; do mv "$f" "$n.txt"; done ```
- Create a new notebook on the Spark Jupyter server (`https://<cluster-name>.azurehdinsight.net/jupyter`).

### 4.2 Understand TF-IDF

[TF-IDF](https://en.wikipedia.org/wiki/Tf%E2%80%93idf) is a statistic to determine the relative importance of the words in a set of documents. Is is computed as the product of two statistics, term frequency (`tf`) and inverse document frequency (`idf`). 

Given a word `t`, a document `d` (in this case, a book) and the collection of all documents `D` we can define `tf(t, d)` as the number of times `t` appears in `d`. This gives us some information about the content of a document but because some terms (eg. "the") are so common, term frequency will tend to incorrectly emphasize documents which happen to use the word "the" more frequently, without giving enough weight to the more meaningful terms.

The inverse document frequency `idf(t, D)` is a measure of how much information the word provides, that is, whether the term is common or rare across all documents. It can be computed as:

![idf](https://ethbigdata2017.blob.core.windows.net/exercise05assets/idf.png)

where $|D|$ is the total number of documents and the denominator represents how many documents contain the word $t$ at least once. However, this would cause a division-by-zero exception if the user query a word that never appear in the dataset. A better formulation would be:

![idf](https://ethbigdata2017.blob.core.windows.net/exercise05assets/idf_smooth.png)

Then, the `tdidf(t, d, D)` is calculated as follows:

![idf](https://ethbigdata2017.blob.core.windows.net/exercise05assets/tfidf.png)

A high weight in `tfidf` is reached by a high term frequency (in the given document) and a low document frequency of the term in the whole collection of documents.

Having already implemented TF-IDF last week in pseudocode, in this week we are going to implement it in Spark. The following code snippet imports the whole dataset into an RDD.

In [ ]:
# sc is automatically defined as SparkContext
# docs will be an RDD in the format [(docName, content)]
docs = sc.wholeTextFiles("/<path-to-gutemberg-folder>/*.txt", minPartitions=100)

### TF-IDF solution code

In [ ]:
# partition the whole dataset into (title, text) pairs
# we assume that the files are uploaded into the /tmp/ folder
docs = sc.wholeTextFiles(
    "wasb:///tmp/*.txt", minPartitions=100).map(
    lambda a: (a[0].split("/tmp/", 1)[1], a[1]))

# count the number of documents
doc_number = docs.count()

# function that get a text and tokenizes it
import re
def tokenize(s):
  return re.split("\\W+", s.lower())

# tokenize the text in every document
tokenized_text = docs.map(lambda a: (a[0], tokenize(a[1])))

# count words in every document
term_frequency = tokenized_text.flatMapValues(lambda x: x).countByValue()

# calculate the document frequency
document_frequency = tokenized_text.flatMapValues(
                     lambda x: x).distinct().map(
                     lambda a: (a[1] ,a[0])).countByKey()

# compute tf-idf
import numpy as np

def tf_idf(doc_num, term_frequency, document_frequency):
    result = []
    for key, value in term_frequency.items():
        doc = key[0]
        term = key[1]
        df = document_frequency[term]
        if (df>0):
          tf_idf = float(value)*np.log(do_num/df)
        
        result.append({"doc":doc, "score":tf_idf, "term":term})
    return result

tf_idf_output = tf_idf(number_of_docs, term_frequency, document_frequency)

# display the first 100 terms
tf_idf_output[:100]